# Digit recognition problems

In [13]:
import math
import numpy as np

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier

import nn
from optim import SGDOptimizer

from supervised_learning import MyMLPClassifier
from dataset.load_data import sklearn_to_df, prepare_data_loader

In [14]:
def prepare_data():
    X, Y = sklearn_to_df(load_digits())
    X_tr, X_te, y_tr, y_te = train_test_split(X, Y, test_size=0.33, random_state=42)
    scaler = MinMaxScaler()
    scaler.fit(X_tr)
    X_tr = scaler.transform(X_tr)
    X_te = scaler.transform(X_te)

    print ("train_x's shape: " + str(X_tr.shape))
    print ("test_x's shape: " + str(X_te.shape))
    print ("train_y's shape: " + str(y_tr.shape))
    print ("test_y's shape: " + str(y_te.shape))
    print()
    
    return X_tr, X_te, y_tr, y_te

def prepare_trainer(model):
    optimizer = SGDOptimizer(model, learning_rate=0.2, regularization=0.015)
    loss_func = nn.CrossEntropyLoss()
    return optimizer, loss_func

def build_model(n_in, n_classes):
    np.random.seed(101)
    model = MyMLPClassifier(n_input=n_in, hiddens=[200, 100], n_output=n_classes, activation='relu')   
    return model

In [15]:
def main():
    X_tr, X_te, y_tr, y_te = prepare_data()
    n_in, n_class = X_tr.shape[1], 10

    num_epochs = 250
    batch_size = 32

    model = build_model(n_in, n_class)
    optimizer, loss_func = prepare_trainer(model)

    model.info()

    # Training loop
    for epoch in range(num_epochs):
        # Prepare the data loader for training data
        data_loader = prepare_data_loader(X_tr, y_tr, batch_size)
        
        # Initialize counters for tracking training progress
        step = 0
        total_loss, total_correct = 0, 0
        total_sample = 0

        for batch_X, batch_y in data_loader:
            # forward pass: compute logits and loss
            batch_logit = model.forward(batch_X)        # output model: logit
            loss = loss_func.forward(batch_logit, batch_y)

            # backward pass and an optimization step
            optimizer.zero_grad()
            dout = loss_func.backward()
            model.backward(dout)
            optimizer.step()

            # log training progress
            step += 1
            total_loss += loss
            batch_yp = np.argmax(batch_logit, axis=1)   # logit --> label
            total_correct += np.sum(batch_yp == batch_y)
            total_sample += len(batch_y)
        print(f"Epoch: {epoch}, loss={total_loss / total_sample:.4f}, train_acc={total_correct / total_sample:.4f}")

    model.eval()

    ypred = np.argmax(model.forward(X_te), axis=1)
    print("\nClassification report of my MLP model:\n", classification_report(y_te, ypred))

    
    skmodel = MLPClassifier()
    skmodel.fit(X_tr, y_tr)

     # Print information about the trained scikit-learn MLPClassifier
    print("\nScikit-learn MLPClassifier Info:")
    print("Number of layers:", skmodel.n_layers_)
    print("Number of neurons in each layer:", skmodel.hidden_layer_sizes)
    print("Number of output classes:", skmodel.n_outputs_)
    print("Activation function:", skmodel.activation)
    print("Solver:", skmodel.solver)
    print("Learning rate:", skmodel.learning_rate)
    print("Initial learning rate:", skmodel.learning_rate_init)
    print("Batch size:", skmodel.batch_size)
    print("Maximum number of iterations:", skmodel.max_iter)
    # Add more model-specific information as needed

    ypred = skmodel.predict(X_te)
    print("\nClassification report of sklearn's model:\n", classification_report(y_te, ypred))

In [16]:
if __name__ == "__main__":
    main()

train_x's shape: (1203, 64)
test_x's shape: (594, 64)
train_y's shape: (1203,)
test_y's shape: (594,)

MyMLPClassifier(
(linear): Linear(in_features=64, out_features=200, bias=True)
(relu): ReLU()
(linear): Linear(in_features=200, out_features=100, bias=True)
(relu): ReLU()
(linear): Linear(in_features=100, out_features=10, bias=True)
)
Epoch: 0, loss=1.9851, train_acc=0.4613
Epoch: 1, loss=1.6108, train_acc=0.7473
Epoch: 2, loss=1.3524, train_acc=0.8113
Epoch: 3, loss=1.1574, train_acc=0.8429
Epoch: 4, loss=1.0099, train_acc=0.8687
Epoch: 5, loss=0.8985, train_acc=0.8861
Epoch: 6, loss=0.8132, train_acc=0.8961
Epoch: 7, loss=0.7466, train_acc=0.8994
Epoch: 8, loss=0.6935, train_acc=0.9027
Epoch: 9, loss=0.6502, train_acc=0.9044
Epoch: 10, loss=0.6143, train_acc=0.9094
Epoch: 11, loss=0.5841, train_acc=0.9135
Epoch: 12, loss=0.5582, train_acc=0.9169
Epoch: 13, loss=0.5358, train_acc=0.9185
Epoch: 14, loss=0.5162, train_acc=0.9235
Epoch: 15, loss=0.4989, train_acc=0.9260
Epoch: 16, loss

/home/funny/miniconda3/envs/pytorch-env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
